# Main Notebook
Notebook contains training for model and eventually trained model. It also includes a way to test new images on the model.

The following snippet labels the images and puts the labels in a CSV file.

In [3]:
import os
import pandas as pd

def create_label_files(pos_folder, neg_folder):
    pos_labels = []
    neg_labels = []

    # Positive class (golf ball present)
    for filename in os.listdir(pos_folder):
        if filename.lower().endswith(".jpg"):  # Adjust if using other formats
            pos_labels.append({"filename": filename, "label": 1})

    # Negative class (no golf ball)
    for filename in os.listdir(neg_folder):
        if filename.lower().endswith(".jpg"):  # Adjust if using other formats
            neg_labels.append({"filename": filename, "label": 0})

    pos_df = pd.DataFrame(pos_labels)
    neg_df = pd.DataFrame(neg_labels)

    return pos_df, neg_df

# Folders
pos_folder = r'C:\Users\rogan\Documents\Programming\GBallDNN\GBallSet\1'
neg_folder = r'C:\Users\rogan\Documents\Programming\GBallDNN\GBallSet\0'

# Create label DataFrames
pos_df, neg_df = create_label_files(pos_folder, neg_folder)

# Save DataFrames to CSV files
pos_df.to_csv('pos_labels.csv', index=False)
neg_df.to_csv('neg_labels.csv', index=False)

This snippet pre-processes the images, associating them with their labels and designates training and testing data sets.

In [5]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.model_selection import train_test_split

# Function to load and preprocess images from multiple directories
def load_and_preprocess_images(image_dirs, labels_dfs):
    images = []
    labels = []
    for image_dir, labels_df in zip(image_dirs, labels_dfs):
        for index, row in labels_df.iterrows():
            filename = row['filename']  # Assuming CSV has a 'filename' column
            label = row['label']  # Assuming CSV has a 'label' column (0 or 1)
            img_path = os.path.join(image_dir, filename)
            img = load_img(img_path, target_size=(224, 224))  # Adjust target_size as needed
            img = img_to_array(img)
            img = preprocess_input(img)
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

# Define the image directories
pos_folder = r'C:\Users\rogan\Documents\Programming\GBallDNN\GBallSet\1'
neg_folder = r'C:\Users\rogan\Documents\Programming\GBallDNN\GBallSet\0'
image_dirs = [pos_folder, neg_folder]

# Load the separate label CSV files into DataFrames
pos_labels_df = pd.read_csv('pos_labels.csv')
neg_labels_df = pd.read_csv('neg_labels.csv')

# Put the DataFrames into a list
labels_dfs = [pos_labels_df, neg_labels_df]

# Load images and labels
images, labels = load_and_preprocess_images(image_dirs, labels_dfs)

# Split data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


The following snippets setup and train the model.

In [6]:
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load pre-trained InceptionV3 model without top layers
base_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add your custom classifier layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
# Combine base model and custom layers into a new model
model = Model(inputs=base_model.input, outputs=predictions)

In [17]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.save('GBallDNN.keras')  # Save as keras file

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

Epoch 1/100
4/4 [==============================] - 9s 983ms/step - loss: 0.2847 - accuracy: 0.9333 - val_loss: 0.6765 - val_accuracy: 0.8667
Epoch 2/100
4/4 [==============================] - 2s 413ms/step - loss: 0.5298 - accuracy: 0.8583 - val_loss: 1.4724 - val_accuracy: 0.6667
Epoch 3/100
4/4 [==============================] - 2s 456ms/step - loss: 0.1854 - accuracy: 0.9000 - val_loss: 0.3724 - val_accuracy: 0.8000
Epoch 4/100
4/4 [==============================] - 2s 426ms/step - loss: 0.0817 - accuracy: 0.9667 - val_loss: 0.0640 - val_accuracy: 1.0000
Epoch 5/100
4/4 [==============================] - 2s 416ms/step - loss: 0.0806 - accuracy: 0.9667 - val_loss: 0.2927 - val_accuracy: 0.8667
Epoch 6/100
4/4 [==============================] - 2s 432ms/step - loss: 0.0734 - accuracy: 0.9750 - val_loss: 0.5341 - val_accuracy: 0.8000
Epoch 7/100
4/4 [==============================] - 2s 408ms/step - loss: 0.0535 - accuracy: 0.9833 - val_loss: 0.1964 - val_accuracy: 0.8667
Epoch 8/100
4

# Test Model
Code to process a new image test model.

In [14]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Example of loading and preprocessing a single new image
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = img_array.reshape((1,) + img_array.shape)  # Add batch dimension
    return img_array

# Example usage:
new_image_path = r'C:\Users\rogan\Documents\Programming\GBallDNN\GBallSet\pracpred\pracnoball.jpg'
target_size = (224, 224)  # Adjust according to your model's input size

preprocessed_image = preprocess_image(new_image_path, target_size)

# Assuming `model` is your trained Keras model
predictions = model.predict(preprocessed_image)

# Assuming binary classification (0 or 1)
predicted_class = predictions[0][0]  # Accessing the first prediction (single image, single class)

print(predicted_class)

if predicted_class < 0.5:
    print("Predicted: Golf ball present")
else:
    print("Predicted: No golf ball")

1/1 [==============================] - 0s 250ms/step
1.0
Predicted: No golf ball
